## This notebook we are implementing the dataframe to save the decisions made by round:

We will be saving the data from the P2 perspective (self=P2, opponent=P1), and the "Win" column plans be filled at the end of each game. 

In [42]:
#Import statements
import time
import sys
import os
import random
# import gym
# from gym import Env
import numpy as np
# from gym.spaces import Discrete
from enum import Enum
from statistics import mean
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
#import Decision_Matrix as dm
import Kenzie_Strategy as dm
import pandas as pd
import statsmodels.api as sm

In [43]:
#Battlefield setup

class Player():
    def __init__(self,name, decksize=60, num_creatures = 20, num_spells = 20) :
       self.name = name
       self.life = 20
       self.decksize = decksize
       self.battlefield = []
       self.deck = self.build_deck(num_creatures,num_spells)
       self.hand = []
       self.landforturn = False
       self.opponent= None 
        
    def set_opponent(self, opponent):
        self.opponent = opponent
        
        
    def build_deck(self,num_creatures,num_spells):
        deck = []
        for i in range(num_creatures):
            deck.append('creature')
        for i in range(num_spells):
            deck.append('spell')
        for i in range(self.decksize - num_creatures - num_spells):
            deck.append('land')
        return deck
    
    def draw(self, number_draw = 1):
        try:
            for i in range(number_draw):
                self.hand.append(self.deck.pop(0))
        except IndexError:
            pass
            #print("Deck size", len(self.deck), "Draw Error")
            
    def attack(self, num_attackers):
        tapper = 0
        for k in range(len(self.battlefield)):
            if self.battlefield[k] == "creature":
                if tapper < num_attackers:
                    self.battlefield[k] = "tapped_creature"
                    tapper += 1
                    
    def block(self, num_block, num_attack):
        counter = 0
        for k in range(num_block):
            if k+1 <= num_attack:
                self.opponent.battlefield.remove("tapped_creature")
                self.battlefield.remove("creature")
        if num_attack >= num_block:
            damage = (num_attack - num_block) * 2
            self.life -= damage
            #print(f"{self.name} lost {damage} life.")
        else:
            pass
            #print("overblocked")
            
    def play(self,  card_choice, df, target_choice = 0):
                   
        if card_choice == 'creature':
            manapool = 0
            tapper = 0
            for k in range(len(self.battlefield)):
                if self.battlefield[k] == 'land':
                    if tapper < 2:
                        self.battlefield[k] = 'tapped_land'
                        tapper += 1
            self.battlefield.append(card_choice)
            if self == player2:
                df.at[(i+1,game_round), 'CP'] += 1
        elif card_choice == 'spell': #spell
            # Play a spell, target choices are [p1, p2, p1 u_c, p1 t_c, p2 u_c, p2 t_C]
            # going from 1-6
            if self == player2:
                df.at[(i+1,game_round), 'ST'].append(target_choice)
            self.play_spell(target_choice=target_choice, df=df)
        elif card_choice == 'land':
            if self == player2:
                df.at[(i+1,game_round), 'LFT'] = True
            # game_record['LFT'] = True fix this
            #if self.player1.landforturn == True:
                #print("You have already played a land for turn")
                #reward(penalty)
                #choose_card_choice([1])
            self.battlefield.append(card_choice)
            self.landforturn = True
            #if game.player2.landforturn == True:
                #df.at[(i+1,game_round), 'LFT'] = True
        self.hand.remove(card_choice)
        return df
            
    def play_spell(self,target_choice,df):
        tapper = 0        
        for k in range(len(self.battlefield)):
            if self.battlefield[k] == 'land':
                 if tapper < 1:
                    self.battlefield[k] = 'tapped_land'
                    tapper += 1
            # spell effects
            # Based on choice, remove that amount of life or that kind of creature from
            # appopriate battlefield
        if target_choice == "self":
            
            self.life = self.life - 3
            #print(f"{self.name}'s life total was targeted by a spell.")
        elif target_choice == "opponent":
            self.opponent.life = self.opponent.life - 3
            #print(f"{self.opponent.name}'s life total was targeted by a spell.")
        elif target_choice == "self_untapped_creature":
            if "creature" in self.battlefield: 
                self.battlefield.remove("creature")
            #print(f"{self.name}'s untapped creature was targeted by a spell.")
        elif target_choice == "self_tapped_creature":
            if "tapped_creature" in self.battlefield: 
                self.battlefield.remove("tapped_creature")
            #print(f"{self.name}'s tapped creature was targeted by a spell.")
        elif target_choice == "opponent_untapped_creature":
            if "creature" in self.opponent.battlefield: 
                self.opponent.battlefield.remove("creature")
            #print(f"{self.opponent.name}'s untapped creature was targeted by a spell.")
        elif target_choice == "opponent_tapped_creature":
            if "tapped_creature" in self.opponent.battlefield:
                self.opponent.battlefield.remove("tapped_creature")
            #print(f"{self.opponent.name}'s tapped creature was targeted by a spell.")
        else:
            pass
            #print("Target acquired " ,target_choice)
        return df

    def turn(self):
        self.landforturn = False
        for k in range(len(self.battlefield)):
            if self.battlefield[k] == "tapped_creature":
                    self.battlefield[k] = "creature"
            if self.battlefield[k] == "tapped_land":
                    self.battlefield[k] = "land"
        self.draw()
        
            
            
class magicgame():
    def __init__(self,player1, player2):
        self.player1 = player1
        self.player2 = player2
        random.shuffle(self.player1.deck)
        random.shuffle(self.player2.deck)
        self.player1.draw(7)    
        self.player2.draw(7)
        #phases for each player will be main phase 1, attacker phase, and blocking phase (1, 2, 3)
        self.phase = 1
        
    #Game Set up

    
    def show_battlefield(self):
        #print("\n __________________________________________________________")
        #print("\n", self.player2.hand)
        #print("\n _____ P2 _____ Life: ", self.player2.life)
        for k in range(len(self.player2.battlefield)):
            #L = untapped land
            if self.player2.battlefield[k] == "land":
                #print("L", end = " ")
                pass
            #l = tapped land
            elif self.player2.battlefield[k] == "tapped_land":
                #print("l", end = " ")
                pass
        #print("\n")
        for k in range(len(self.player2.battlefield)):
            #C = untapped creature
            if self.player2.battlefield[k] == "creature":
                #print("C", end = " ")
                pass
            #c = tapped creature
            elif self.player2.battlefield[k] == "tapped_creature":
                #print("c", end = " ")
                pass
        #print("\n")
        for k in range(len(self.player1.battlefield)):
            if self.player1.battlefield[k] == "creature":
                #print("C", end = " ")
                pass
            elif self.player1.battlefield[k] == "tapped_creature":
                pass
                #print("c", end = " ")
        #print("\n")
        for k in range(len(self.player1.battlefield)):
            if self.player1.battlefield[k] == "land":
                #print("L", end = " ")
                pass
            elif self.player1.battlefield[k] == "tapped_land":
                #print("l", end = " ")
                pass
        #print("\n _____ P1 _____ Life: ", self.player1.life)
        #print("\n", self.player1.hand)
        
    
    def nextphase(self):
        self.phase += 1
        if self.phase == 4:
            self.phase = 11
        elif self.phase == 14:
            self.phase = 1
        else:
            pass
        
    def show_winner(p1, p2, deck, df):
        #print("P1 life: " + str(p1))
        #print("P2 life: " + str(p2))
        #print("P1 Deck size", len(player1.deck))
        #print("P2 Deck size", len(player2.deck))
        winner = ""
        if p1 < 1:
            #print("P1 loses, P2 wins")
            winner = "P2"
            AI_win = True
        elif p2 < 1:
            #print("P2 loses, P1 wins")
            winner = "P1"
            AI_win = False
        elif len(deck) < 0:
            #print("P2 has no more cards in library, P1 wins")
            winner = "P1"
            AI_win = False
        else:
            pass
        df.at[(i+1), 'Win']= AI_win
        return winner, AI_win, df       

In [44]:
#Player Actions

def chooseAction(value, prob):
        decision = np.random.choice(value, p = prob)
        return decision

In [45]:
def playgame(player1,player2, i, df):
    game = magicgame(player1,player2)
    
    global game_round 
    game_round= 1
    AI_win = None
    df = df.append(pd.Series(np.nan,index=df.columns, name=(i+1,game_round)))
    df.at[(i+1,game_round), 'CP'] = 0
    df.at[(i+1,game_round), 'ST'] = []
    df.at[(i+1,game_round), 'LFT'] = False
    while (game.player1.life > 0 and game.player2.life > 0) and (len(game.player2.deck) > 0):
        
        #game_record.append({'Game': i, 'Round': game_round, "LFT": False,"CP": 0,"SLT": 0, "OLT": 0,"ST": [],"SUC":0,"OUC": 0,"STC":0,"OTC":0,"SAC":0,"OAC":0,"SBC":0,"OBC":0, 'Win':None})
        # p1 phase 1
        #call creature tracker, set output = previous_state_tup
        if game.phase == 1:
            # show the battle field
            game.show_battlefield()
            #print("What does P1 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2, \
                #P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),\
                     #2 = land, 8 = pass phase")
            
            numConv = {"creature":0, 
                        "spell":1, 
                         "land":2, 
                          "P1 T creature":4,
                          "P2 U creature":5, 
                         "P2 T creature": 6,
                          "pass phase":8}
            
            p1_decision = None
            if game.player1.battlefield.count("land") >= 2:
                if game.player1.landforturn == False:
                    if "land" in game.player1.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        if ("creature" in game.player1.hand) and ("spell" not in game.player1.hand):
                            values.append("creature")
                            prob = [0.7,0.05,0.25]
                        elif ("spell" in game.player1.hand) and ("creature" not in game.player1.hand):
                            values.append("spell")
                            prob = [0.7,0.05,0.25]
                        elif ("spell" in game.player1.hand) and "creature" in game.player1.hand:
                            values.append("spell")
                            values.append("creature")
                            prob = [0.7,0.1,0.1,0.1]
                        p1_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.player1.hand))
                        num_choices.append("pass phase")
                        p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.player1.hand).difference(set(["land"])))
                    num_choices.append("pass phase")
                    p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            elif game.player1.battlefield.count("land") == 1:
                if game.player1.landforturn == False:
                    if "land" in game.player1.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        if ("spell" in game.player1.hand):
                            values.append("spell")
                            prob = [0.7,0.05,0.25]
                        else:
                            pass
                            #print("You only have one untapped mana. You can't play a creature and you don't have a spell.")
                        p1_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.player1.hand).difference(set(["creature"])))
                        num_choices.append("pass phase")
                        p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.player1.hand).difference(set(["creature","land"])))
                    num_choices.append("pass phase")
                    p1_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            else:
                if game.player1.landforturn == False:
                    if "land" in game.player1.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        p1_decision = chooseAction(values,prob)
                    else:
                        p1_decision = "pass phase"
                else:
                    p1_decision = "pass phase"
                    #print("No more moves. Pass the turn")
            
            p2_decision = None
            
        # Stopped here

            # If player one plays a spell
            if p1_decision == "spell":
                # Player one choose target
                #print("Target?")
                #######
                ## These numbers needs to be discussed
                values = ["self","opponent"]
                probs = [.05,.95]
                p1_set_creatures = set(game.player1.battlefield).difference(set(["land","tapped_land"]))
                #print(p1_set_creatures, "P1 valid targets")
                p2_set_creatures = set(game.player2.battlefield).difference(set(["land","tapped_land"]))
                #print(p2_set_creatures, "P2 valid targets")
                if len(p1_set_creatures) != 0 and len(p2_set_creatures) == 0:
                    probs = [.05, .85]
                    temp_probs = [.1/len(p1_set_creatures)] * len(p1_set_creatures)
                    probs += temp_probs
                    if "creature" in game.player1.battlefield:
                        values.append("self_untapped_creature")
                    if "tapped_creature" in game.player1.battlefield:
                        values.append("self_tapped_creature")
                elif len(p1_set_creatures) == 0 and len(p2_set_creatures) != 0:
                    probs = [.05]
                    temp_probs = [.95/(len(p2_set_creatures)+1)] * (len(p2_set_creatures)+1)
                    probs += temp_probs
                    if "creature" in game.player2.battlefield:
                        values.append("opponent_untapped_creature")
                    if "tapped_creature" in game.player2.battlefield:
                        values.append("opponent_tapped_creature")
                elif len(p1_set_creatures) != 0 and len(p2_set_creatures) != 0:
                    probs = [.05]
                    temp = .85/(len(p2_set_creatures)+1)
                    probs.append(temp)
                    temp_probs_p1 = [.1/len(p1_set_creatures)]*len(p1_set_creatures)
                    probs += temp_probs_p1
                    temp_probs_p2 = [temp]*len(p2_set_creatures)
                    probs += temp_probs_p2
                    if "creature" in game.player1.battlefield:
                        values.append("self_untapped_creature")
                    if "tapped_creature" in game.player1.battlefield:
                        values.append("self_tapped_creature")
                    if "creature" in game.player2.battlefield:
                        values.append("opponent_untapped_creature")
                    if "tapped_creature" in game.player2.battlefield:
                        values.append("opponent_tapped_creature")
                #######
                target = np.random.choice(values, p = probs) #P1 wants to target P2 stuff
                game.player1.play(card_choice=p1_decision, target_choice=target, df=df)
            elif p1_decision == "pass phase":
                game.nextphase()
            else:
                game.player1.play(card_choice = p1_decision, df=df)
                
        
        elif game.phase == 2:
            game.show_battlefield()
            #print("How many P1 attackers? Any N attackers, 0 is pass")
            #global num_attack
            #print(f'{game.player1.battlefield.count("creature")} p1 battlefield')
            num_attackers = list(range(0,game.player1.battlefield.count("creature")+1))
            probs = [1/(game.player1.battlefield.count("creature")+2)]*(game.player1.battlefield.count("creature"))
            probs.append(2/(game.player1.battlefield.count("creature")+2))
            num_attack = np.random.choice(num_attackers, p = probs) # needs to be random
            #print("Player 1 is attacking with " + str(num_attack) + " creatures.")
            game.player1.attack(num_attack)
            df.at[(i+1,game_round), 'OAC'] = num_attack
            game.nextphase()
            
        elif game.phase == 3:
            if num_attack == 0:
                df.at[(i+1,game_round), 'SBC']= 0
                pass
            else:
                game.show_battlefield()
                #print("How many P2 blockers? Any N blockers, 0 is pass")
                values = list(range(0,game.player2.battlefield.count("creature")+1))
                probs = [1/(game.player2.battlefield.count("creature")+2)]*(game.player2.battlefield.count("creature"))
                probs.append(2/(game.player2.battlefield.count("creature")+2))
                p2_decision = np.random.choice(values,p = probs) # needs to be random
                #print("Player 2 is blocking with " + str(p2_decision) + " creatures.")
                game.player2.block(p2_decision, num_attack)
                df.at[(i+1,game_round), 'SBC'] = p2_decision
            game.player2.turn()
            game.nextphase()
            
        elif game.phase == 11:
            game.show_battlefield()
            #print("What does P2 want to play? 0 = creature, 1 = spell (and target of P1 = 1, P2 = 2, \
                #P1 U creature = 3, P1 T creature = 4, P2 U creature = 5, P2 T creature = 6),\
                     #2 = land, 8 = pass phase")
            
            p2_decision = None
            if game.player2.battlefield.count("land") >= 2:
                if game.player2.landforturn == False:
                    if "land" in game.player2.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        if ("creature" in game.player2.hand) and ("spell" not in game.player2.hand):
                            values.append("creature")
                            prob = [0.7,0.05,0.25]
                        elif ("spell" in game.player2.hand) and ("creature" not in game.player2.hand):
                            values.append("spell")
                            prob = [0.7,0.05,0.25]
                        elif ("spell" in game.player2.hand) and "creature" in game.player2.hand:
                            values.append("spell")
                            values.append("creature")
                            prob = [0.7,0.1,0.1,0.1]
                        p2_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.player2.hand))
                        num_choices.append("pass phase")
                        p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.player2.hand).difference(set(["land"])))
                    num_choices.append("pass phase")
                    p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            elif game.player2.battlefield.count("land") == 1:
                if game.player2.landforturn == False:
                    if "land" in game.player2.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        if ("spell" in game.player2.hand):
                            values.append("spell")
                            prob = [0.7,0.05,0.25]
                        else:
                            pass
                            #print("You only have one untapped mana. You can't play a creature and you don't have a spell.")
                        p2_decision = chooseAction(values,prob)
                    else:
                        num_choices = list(set(game.player2.hand).difference(set(["creature"])))
                        num_choices.append("pass phase")
                        p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
                else:
                    num_choices = list(set(game.player2.hand).difference(set(["creature","land"])))
                    num_choices.append("pass phase")
                    p2_decision = chooseAction(num_choices,[1/len(num_choices)]*len(num_choices))
            else:
                if game.player2.landforturn == False:
                    if "land" in game.player2.hand:
                        values  = ["land","pass phase"]
                        prob = [0.95,0.05]
                        p2_decision = chooseAction(values,prob)
                    else:
                        p2_decision = "pass phase"
                else:
                    p2_decision = "pass phase"
                    #print("No more moves. Pass the turn")

            # If player one plays a spell
            if p2_decision == "spell":
                # Player one choose target
                #print("Target?")
                #######
                ## These numbers needs to be discussed
                values = ["self","opponent"]
                probs = [.05,.95]
                p1_set_creatures = set(game.player1.battlefield).difference(set(["land","tapped_land"]))
                #print(p1_set_creatures, "P1 valid targets")
                p2_set_creatures = set(game.player2.battlefield).difference(set(["land","tapped_land"]))
                #print(p2_set_creatures, "P2 valid targets")
                if len(p2_set_creatures) != 0 and len(p1_set_creatures) == 0:
                    probs = [.05, .85]
                    temp_probs = [.1/len(p2_set_creatures)] * len(p2_set_creatures)
                    probs += temp_probs
                    if "creature" in game.player2.battlefield:
                        values.append("self_untapped_creature")
                    if "tapped_creature" in game.player2.battlefield:
                        values.append("self_tapped_creature")
                elif len(p2_set_creatures) == 0 and len(p1_set_creatures) != 0:
                    probs = [.05]
                    temp_probs = [.95/(len(p1_set_creatures)+1)] * (len(p1_set_creatures)+1)
                    probs += temp_probs
                    if "creature" in game.player1.battlefield:
                        values.append("opponent_untapped_creature")
                    if "tapped_creature" in game.player1.battlefield:
                        values.append("opponent_tapped_creature")
                elif len(p2_set_creatures) != 0 and len(p1_set_creatures) != 0:
                    probs = [.05]
                    temp = .85/(len(p1_set_creatures)+1)
                    probs.append(temp)
                    temp_probs_p2 = [.1/len(p2_set_creatures)]*len(p2_set_creatures)
                    probs += temp_probs_p2
                    temp_probs_p1 = [temp]*len(p1_set_creatures)
                    probs += temp_probs_p1
                    if "creature" in game.player2.battlefield:
                        values.append("self_untapped_creature")
                    if "tapped_creature" in game.player2.battlefield:
                        values.append("self_tapped_creature")
                    if "creature" in game.player1.battlefield:
                        values.append("opponent_untapped_creature")
                    if "tapped_creature" in game.player1.battlefield:
                        values.append("opponent_tapped_creature")
                #######
                target = np.random.choice(values, p = probs) #P1 wants to target P2 stuff
                game.player2.play(card_choice=p2_decision, target_choice=target, df=df)
            elif p2_decision == "pass phase":
                game.nextphase()
            else:
                game.player2.play(card_choice=p2_decision, df=df)
                
        elif game.phase == 12:
            game.show_battlefield()
            #print("How many P2 attackers? Any N attackers, 0 is pass")
            #global num_attack
            #print(f'{game.player2.battlefield.count("creature")} p2 battlefield')
            num_attackers = list(range(0,game.player2.battlefield.count("creature")+1))
            probs = [1/(game.player2.battlefield.count("creature")+2)]*(game.player2.battlefield.count("creature"))
            probs.append(2/(game.player2.battlefield.count("creature")+2))
            num_attack = np.random.choice(num_attackers, p = probs) # needs to be random
            df.at[(i+1,game_round), 'SAC'] = num_attack

# Note we are choosing to look at the opponents creatures + tapped_creatures on the battefeild
            #num_attack = int(dm.decision(dm.count_array, self_creatures=game.player2.battlefield.count("creature"), opponent_creatures=game.player1.battlefield.count("creature") + game.player1.battlefield.count("tapped_creature"), self_life=game.player2.life, opponent_life=game.player1.life))
            #print("Player 2 is attacking with " + str(num_attack) + " creatures.")
            game.player2.attack(num_attack)
            game.nextphase()
        elif game.phase == 13:
            
            if num_attack == 0:
                df.at[(i+1,game_round), 'OBC']= 0
                df.at[(i+1,game_round), 'SLT']= game.player2.life
                df.at[(i+1,game_round), 'OLT']= game.player1.life
                df.at[(i+1,game_round), 'SUC']= game.player2.battlefield.count("creature")
                df.at[(i+1,game_round), 'OUC']= game.player1.battlefield.count("creature")
                df.at[(i+1,game_round), 'OTC']= game.player1.battlefield.count("tapped_creature")
                df.at[(i+1,game_round), 'STC']= game.player2.battlefield.count("tapped_creature")
                pass
            else:
                game.show_battlefield()
                #print("How many P1 blockers? Any N blockers, 0 is pass")
                values = list(range(0,game.player1.battlefield.count("creature")+1))
                probs = [1/(game.player1.battlefield.count("creature")+2)]*(game.player1.battlefield.count("creature"))
                probs.append(2/(game.player1.battlefield.count("creature")+2))
                p1_decision = np.random.choice(values,p = probs) # needs to be random
                #print("Player 1 is blocking with " + str(p1_decision) + " creatures.")
                df.at[(i+1,game_round), 'OBC']= p1_decision
                game.player1.block(p1_decision, num_attack)
            df.at[(i+1,game_round), 'SLT']= game.player2.life
            df.at[(i+1,game_round), 'OLT']= game.player1.life
            df.at[(i+1,game_round), 'SUC']= game.player2.battlefield.count("creature")
            df.at[(i+1,game_round), 'OUC']= game.player1.battlefield.count("creature")
            df.at[(i+1,game_round), 'OTC']= game.player1.battlefield.count("tapped_creature")
            df.at[(i+1,game_round), 'STC']= game.player2.battlefield.count("tapped_creature")
            #initializing next round
            game_round += 1
            df = df.append(pd.Series(np.nan,index=df.columns, name=(i+1,game_round)))
            df.at[(i+1,game_round), 'CP'] = 0
            df.at[(i+1,game_round), 'ST'] = []
            df.at[(i+1,game_round), 'LFT'] = False
            game.nextphase()
            game.player1.turn()
        
    winner, AI_win, df = magicgame.show_winner(game.player1.life, game.player2.life, game.player2.deck, df)
    # print(df)
    # print(df['CP'].value_counts())
    winners.append(winner)
    return df

In [46]:
#dm.decision_array
game_record = pd.DataFrame(columns = ["Game","Round","LFT","CP","SLT", "OLT","ST","SUC","OUC","STC","OTC","SAC","OAC","SBC","OBC", 'Win'])
game_record.astype(dtype = {'Game': int, 'Round': int, "LFT": bool, "CP" : int, "SLT": int, "OLT": int , "SUC": int,"OUC": int,"STC": int,"OTC": int,"SAC": int,"OAC": int,"SBC": int,"OBC": int, "Win": bool})
game_record.set_index(["Game","Round"], inplace = True)
game_history = pd.DataFrame(columns = ["Game","Round","LFT","CP","SLT", "OLT","ST","SUC","OUC","STC","OTC","SAC","OAC","SBC","OBC", 'Win'])
game_history.astype(dtype = {'Game': int, 'Round': int, "LFT": bool, "CP" : int, "SLT": int, "OLT": int , "SUC": int,"OUC": int,"STC": int,"OTC": int,"SAC": int,"OAC": int,"SBC": int,"OBC": int, "Win": bool})
game_history.set_index(["Game","Round"], inplace = True)
win_ratio = []
#num_decisions = []
random.seed(0)
start_time = time.time()
for j in range(1):
    winners = []
    #dm.count_array = np.full((21,21,21),0)
    for i in range(10000):
        #print("Game", i+1)
        player1 = Player('player 1')
        player2 = Player('player 2')
        player1.set_opponent(player2)
        player2.set_opponent(player1)
        temp_df = playgame(player1,player2, i, df=game_record)
        game_history = pd.concat([game_history,temp_df])
        #num_decisions.append(dm.count_array.sum())
    winner = max(winners)
    temp = (winner == 'P2')
    #dm.reward(dm.decision_array, dm.count_array, winner=temp)
    win_ratio.append(winners.count('P2')/len(winners))
end_time = time.time()
# print(game_history)
print("total time taken this loop: ", end_time - start_time)

total time taken this loop:  761.2398331165314


In [47]:
game_history

LFT CP  SLT  OLT                          ST  SUC  OUC  STC  \
Game  Round                                                                  
1     1       True  0   20   20                          []    0    0    0   
      2       True  0   17   14        [opponent, opponent]    0    0    0   
      3      False  0   17   14                          []    0    0    0   
      4      False  0   11   14                          []    0    0    0   
      5      False  0    7   14                          []    0    0    0   
...            ... ..  ...  ...                         ...  ...  ...  ...   
10000 8      False  0   15   12                          []    0    0    0   
      9       True  0   13   12  [opponent_tapped_creature]    0    0    0   
      10      True  3    3    9                  [opponent]    3    0    0   
      11      True  1    3    9                          []    3    1    0   
      12     False  0  NaN  NaN                          []  NaN  NaN  NaN   

             OTC  SAC  OAC  SBC  OBC    Win  
Game  Round                                  
1     1        0    0    0    0    0  False  
      2        0    0    0    0    0  False  
      3        0    0    0    0    0  False  
      4        0    0    0    0    0  False  
      5        2    0    2    0    0  False  
...          ...  ...  ...  ...  ...    ...  
10000 8        1    0    1    0    0  False  
      9        0    0    1    0    0  False  
      10       2    0    2    0    0  False  
      11       0    0    1    1    0  False  
      12     NaN  NaN  NaN  NaN  NaN  False  

[89090 rows x 14 columns]

In [48]:
# Standardizing df

game_history.fillna('0', inplace=True)
#Lst = ['opponent', 'opponent_untapped_creature', 'opponent_tapped_creature', 'self', 'self_untapped_creature', 'self_tapped_creature']
#game_history = pd.concat([game_history,game_history.ST.str.findall('|'.join(Lst)).str[0].str.get_dummies()], axis=1)
game_history['ST_Opponent'] = game_history.ST.apply(lambda x: 1 if x.count('opponent')>0 else 0)
game_history['ST_Opponent_UT_Creature'] = game_history.ST.apply(lambda x: 1 if x.count('opponent_untapped_creature')>0 else 0)
game_history['ST_Opponent_T_Creature'] = game_history.ST.apply(lambda x: 1 if x.count('opponent_tapped_creature')>0 else 0)
game_history['ST_self_UT_Creature'] = game_history.ST.apply(lambda x: 1 if x.count('self_untapped_creature')>0 else 0)
game_history['ST_self_T_Creature'] = game_history.ST.apply(lambda x: 1 if x.count('self_tapped_creature')>0 else 0)
game_history['ST_self'] = game_history.ST.apply(lambda x: 1 if x.count('self')>0 else 0)
game_history.drop(columns=['ST'], inplace=True)
game_history.head(20)

LFT  CP SLT OLT SUC OUC STC OTC SAC OAC SBC OBC    Win  \
Game Round                                                             
1    1       True   0  20  20   0   0   0   0   0   0   0   0  False   
     2       True   0  17  14   0   0   0   0   0   0   0   0  False   
     3      False   0  17  14   0   0   0   0   0   0   0   0  False   
     4      False   0  11  14   0   0   0   0   0   0   0   0  False   
     5      False   0   7  14   0   0   0   2   0   2   0   0  False   
     6      False   0   5  14   0   2   0   1   0   1   0   0  False   
     7      False   0   0   0   0   0   0   0   0   3   0   0  False   
2    1      False   0  20  20   0   0   0   0   0   0   0   0  False   
     2       True   0  17  17   0   0   0   0   0   0   0   0  False   
     3       True   0  17  11   0   0   0   0   0   0   0   0  False   
     4       True   0  17  11   0   0   0   0   0   0   0   0  False   
     5       True   0  11   8   0   0   0   0   0   0   0   0  False   
     6      False   1   9   8   1   1   0   1   0   1   0   0  False   
     7       True   0   7   8   0   0   0   1   0   2   1   0  False   
     8      False   0   0   0   0   0   0   0   0   1   0   0  False   
3    1       True   0  20  20   0   0   0   0   0   0   0   0   True   
     2       True   1  17  20   1   0   0   0   0   0   0   0   True   
     3       True   0  14  18   0   0   1   0   1   0   0   0   True   
     4       True   0  12  16   0   0   1   1   1   1   0   0   True   
     5      False   1  10  12   0   2   2   1   2   1   0   0   True   

            ST_Opponent  ST_Opponent_UT_Creature  ST_Opponent_T_Creature  \
Game Round                                                                 
1    1                0                        0                       0   
     2                1                        0                       0   
     3                0                        0                       0   
     4                0                        0                       0   
     5                0                        0                       0   
     6                0                        0                       0   
     7                0                        0                       0   
2    1                0                        0                       0   
     2                1                        0                       0   
     3                1                        0                       0   
     4                0                        0                       0   
     5                1                        0                       0   
     6                0                        0                       0   
     7                0                        0                       0   
     8                0                        0                       1   
3    1                0                        0                       0   
     2                0                        0                       0   
     3                0                        0                       0   
     4                0                        0                       0   
     5                0                        0                       0   

            ST_self_UT_Creature  ST_self_T_Creature  ST_self  
Game Round                                                    
1    1                        0                   0        0  
     2                        0                   0        0  
     3                        0                   0        0  
     4                        0                   0        0  
     5                        0                   0        0  
     6                        0                   0        0  
     7                        0                   0        0  
2    1                        0                   0        0  
     2                        0                   0        0  
     3                        0                   0        0  


In [49]:
# game_history.to_csv("C:/Users/bxg190019/Desktop/mtg_dataset/mtg_dataset.csv")

In [50]:
Y = game_history["Win"].reset_index()
Y.drop(columns = ["Game","Round"], inplace = True)
X = game_history.drop(columns = ["Win","ST_self_T_Creature","SLT","OLT"])

In [51]:
X = X.astype(int)
labels = X.columns
labels

Index(['LFT', 'CP', 'SUC', 'OUC', 'STC', 'OTC', 'SAC', 'OAC', 'SBC', 'OBC',
       'ST_Opponent', 'ST_Opponent_UT_Creature', 'ST_Opponent_T_Creature',
       'ST_self_UT_Creature', 'ST_self'],
      dtype='object')

In [52]:
scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
X_scaled = pd.DataFrame(data = X_scaled, columns = labels)
Y

,Win
0,False
1,False
2,False
3,False
4,False
...,...
89085,False
89086,False
89087,False
89088,False


In [53]:
logit_model = sm.Logit(Y,X_scaled)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.576009
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                    Win   No. Observations:                89090
Model:                          Logit   Df Residuals:                    89075
Method:                           MLE   Df Model:                           14
Date:                Tue, 01 Feb 2022   Pseudo R-squ.:                  0.1688
Time:                        20:29:37   Log-Likelihood:                -51317.
converged:                       True   LL-Null:                       -61736.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
LFT                        -0.0631      0.008     -8.026      0.000      -0.079   

In [54]:
dir(result)
CI = result.conf_int()
CI_length = abs(((CI[0] + CI[1])/2)).sort_values(ascending = False)
# 0.2 cuttoff is okay for now. anything more than 0.2, we can throw out as unreliable

In [55]:
# intervals that contain 0 will get 0 weight
# intervals that are negative will get negative weight
# intervals that are positive with get positive weight
CI_length

OAC                        1.097289
STC                        0.807213
SUC                        0.484303
OUC                        0.460382
SBC                        0.348764
ST_Opponent_T_Creature     0.193274
ST_Opponent                0.162728
OTC                        0.147590
CP                         0.119282
ST_self                    0.108643
ST_Opponent_UT_Creature    0.063873
LFT                        0.063125
ST_self_UT_Creature        0.016181
SAC                        0.011677
OBC                        0.002329
dtype: float64

1) We need to set up the Q-value. Find a cutoff (through some ad-hoc method (we'll have to discuss this))

2) Look up ways to deal with zero-inflated data

- List of Problems

1) Land for Turn (LFT)

2) Creatures Played (CP)

3) Self Tapped Creatures (STC): unreliable due to huge Confidence Interval (CI)

4) Graph of features as they change over iterations (to keep track) : Line Graph

In [56]:
selected_vars_CI_length = CI_length[CI_length <= 0.2]
selected_vars = selected_vars_CI_length.index
selected_vars # less than 0.2

Index(['ST_Opponent_T_Creature', 'ST_Opponent', 'OTC', 'CP', 'ST_self',
       'ST_Opponent_UT_Creature', 'LFT', 'ST_self_UT_Creature', 'SAC', 'OBC'],
      dtype='object')

In [57]:
CI_select = CI.loc[selected_vars]
CI_select = CI_select[~((CI_select[0] <= 0) & (CI_select[1] >= 0))]

In [58]:
CI_select

,0,1
ST_Opponent_T_Creature,0.172099,0.214448
ST_Opponent,0.147737,0.177718
OTC,0.102445,0.192735
CP,-0.141282,-0.097282
ST_self,-0.123895,-0.093391
ST_Opponent_UT_Creature,-0.078696,-0.049050
LFT,-0.078540,-0.047710
ST_self_UT_Creature,-0.031337,-0.001026


In [59]:
SE_select = result.bse.loc[CI_select.index]

In [60]:
ind = []
for i in range(len(SE_select)):
    if CI_select.iloc[i,1] > 0:
        ind.append(1)
    else:
        ind.append(-1)

In [61]:
SE_select = SE_select*ind
weights = 1/SE_select
weights

ST_Opponent_T_Creature      92.562432
ST_Opponent                130.750567
OTC                         43.414695
CP                         -89.090074
ST_self                   -128.506707
ST_Opponent_UT_Creature   -132.225660
LFT                       -127.147208
ST_self_UT_Creature       -129.321810
dtype: float64

In [ ]:
# now we need to create function. weights are taken care off
